<a href="https://colab.research.google.com/github/vrnb/vrnb/blob/main/DATA_BOWL_pt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

# Paths of the CSV files
file_paths = [f'/content/drive/MyDrive/NFL DATA BOWL checkpoints/overlap_{i}_week_7.csv' for i in range(1, 6)]

# Read each CSV file into a DataFrame
data_frames = [pd.read_csv(path) for path in file_paths]

# Concatenate all the DataFrames
combined_df = pd.concat(data_frames)

# Display the combined DataFrame



In [ ]:
qualified = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_tracking_week_7.csv',None)

<ipython-input-166-927064610893>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  qualified = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_tracking_week_7.csv',None)
<ipython-input-166-927064610893>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  qualified = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_tracking_week_7.csv',None)


In [ ]:
import pandas as pd

# Assuming combined_df and qualified are already defined

# Step 1: Check data types
print("Data types in combined_df:")
print(combined_df[['gameId', 'playId', 'frameId', 'playerName']].dtypes)
print("\nData types in qualified:")
print(qualified[['gameId', 'playId', 'frameId', 'displayName']].dtypes)

# Step 2: Convert data types if necessary
# Example: If 'gameId' is int in one DataFrame and string in another, convert both to string
combined_df['gameId'] = combined_df['gameId'].astype(str)
qualified['gameId'] = qualified['gameId'].astype(str)

# Do the same for 'playId', 'frameId', and rename 'displayName' to 'playerName' in 'qualified'
combined_df['playId'] = combined_df['playId'].astype(str)
qualified['playId'] = qualified['playId'].astype(str)

combined_df['frameId'] = combined_df['frameId'].astype(str)
qualified['frameId'] = qualified['frameId'].astype(str)

qualified_renamed = qualified.rename(columns={'displayName': 'playerName'})

# Step 3: Perform the merge
merged_df = pd.merge(combined_df, qualified_renamed, how='left', on=['gameId', 'playId', 'frameId', 'playerName'])

# Display the merged DataFrame


Data types in combined_df:
gameId        object
playId        object
frameId       object
playerName    object
dtype: object

Data types in qualified:
gameId          int64
playId          int64
frameId         int64
displayName    object
dtype: object


In [ ]:
# Filter rows where 'position' is either 'OLB', 'CB', or 'DE'
filtered_df = merged_df[merged_df['position'].isin(['OLB', 'CB', 'DE'])]




In [ ]:
import pandas as pd

# Assuming merged_df and filtered_df are your DataFrames

# Convert 'overlapPercentage' to numeric
filtered_df['overlapPercentage'] = pd.to_numeric(filtered_df['overlapPercentage'], errors='coerce')

# Calculate the cumulative overlap
cumulative_overlap = filtered_df.groupby(['gameId', 'playId', 'playerName'])['overlapPercentage'].sum().reset_index()
cumulative_overlap = cumulative_overlap.rename(columns={'overlapPercentage': 'cumulativeOverlap'})

# Rank players within each gameId and playId based on the cumulativeOverlap
cumulative_overlap['overlapRank'] = cumulative_overlap.groupby(['gameId', 'playId'])['cumulativeOverlap'].rank(ascending=False, method='first')

# Merge the rankings back into the original filtered_df
# First, drop the original cumulativeOverlap from filtered_df to avoid conflicts


# Merge, ensuring each player in each play gets their new rank
filtered_df = pd.merge(filtered_df, cumulative_overlap[['gameId', 'playId', 'playerName', 'cumulativeOverlap', 'overlapRank']], on=['gameId', 'playId', 'playerName'], how='left')


<ipython-input-169-e30100fb13e9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['overlapPercentage'] = pd.to_numeric(filtered_df['overlapPercentage'], errors='coerce')


In [ ]:
nonf = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_runs_week_7.csv')

<ipython-input-193-78b3106f4175>:1: DtypeWarning: Columns (31,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  nonf = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL checkpoints/qualified_runs_week_7.csv')


In [ ]:
nonf['right_left'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'right' and x['RunIntention'] == 'left' else 0, axis=1)
nonf['right_right'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'right' and x['RunIntention'] == 'right' else 0, axis=1)
nonf['left_right'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'left' and x['RunIntention'] == 'right' else 0, axis=1)
nonf['left_left'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'left' and x['RunIntention'] == 'left' else 0, axis=1)

In [ ]:
import pandas as pd

# Step 1: Create a DataFrame for ballcarrier
ballcarrier_df = nonf[nonf['ballcarrier'] == 1].copy()

# Step 2: Calculate ballcarrier_direction_1 and ballcarrier_direction_2
# ballcarrier_direction_1
ballcarrier_df['ballcarrier_direction_1'] = ballcarrier_df.apply(
    lambda row: row.dir + 360 if (row.ToLeft and row.dir < 90) else row.dir,
    axis=1)

# Adjust ballcarrier_direction_1
ballcarrier_df['ballcarrier_direction_1'] = ballcarrier_df.apply(
    lambda row: row.dir - 360 if ((not row.ToLeft) and row.dir > 270) else row.ballcarrier_direction_1,
    axis=1)

# ballcarrier_direction_2
ballcarrier_df['ballcarrier_direction_2'] = ballcarrier_df.apply(
    lambda row: row.ballcarrier_direction_1 - 180 if row.ToLeft else row.ballcarrier_direction_1,
    axis=1)

ballcarrier_df['y_in_5_frames'] = ballcarrier_df.groupby(['gameId', 'playId'])['y'].shift(-5)
ballcarrier_df['direction_in_5_frames'] = ballcarrier_df.groupby(['gameId', 'playId'])['ballcarrier_direction_2'].shift(-5)
ballcarrier_df['difference'] = ballcarrier_df['direction_in_5_frames']-ballcarrier_df['ballcarrier_direction_2']

def calculate_good_job(row):
    if (row['left_left'] == 1 or row['right_left'] == 1) and row['difference'] > 0:
        return 1  # Set to 1 for positive difference in left left/right left
    elif (row['left_right'] == 1 or row['right_right'] == 1) and row['difference'] < 0:
        return 1  # Set to 1 for negative difference in left right/right right
    else:
        return 0  # Default case

# Apply the function to create the new column
ballcarrier_df['good_job'] = ballcarrier_df.apply(calculate_good_job, axis=1)

# Step 3: Merge the new columns back into the original DataFrame 'nonf'
ballcarrier_df = ballcarrier_df.add_prefix('ballcarrier_')

# Step 2: Merge ballcarrier_x and ballcarrier_y into nonf
# Selecting only the relevant columns for merge
columns_to_merge = ['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId', 'ballcarrier_x', 'ballcarrier_y','ballcarrier_y_in_5_frames','ballcarrier_difference','ballcarrier_direction_in_5_frames','ballcarrier_ballcarrier_direction_2','ballcarrier_good_job']

nonf = pd.merge(nonf, ballcarrier_df[columns_to_merge],
                left_on=['gameId', 'playId', 'frameId'],
                right_on=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'],
                how='left')

# Optionally, you can drop the redundant gameId, playId, and frameId columns from ballcarrier_df if they exist
nonf.drop(columns=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'], inplace=True)


# Display


In [ ]:
def is_inside(row):
    # Check the play direction based on specific columns
    if row.left_right == 1 or row.right_left == 1:
        return 1 if row.y < row.ballcarrier_y_in_5_frames else 0
    elif row.left_left == 1 or row.right_right == 1:
        return 1 if row.y > row.ballcarrier_y_in_5_frames else 0
    else:
        return 0  # Default case if no play direction is specified

# Apply the function to each row in nonf
nonf['is_inside'] = nonf.apply(is_inside, axis=1)


In [ ]:
tackles = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/tackles.csv',None)

<ipython-input-197-5ca97894bcfc>:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  tackles = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/tackles.csv',None)
<ipython-input-197-5ca97894bcfc>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  tackles = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/tackles.csv',None)


In [ ]:
import pandas as pd

# Assuming nonf and tackles are your DataFrames

# Step 1: Check and convert data types
# Convert gameId, playId, and nflId to string in both DataFrames (or to int if more appropriate)
nonf['gameId'] = nonf['gameId'].astype(str)
nonf['playId'] = nonf['playId'].astype(str)
nonf['nflId'] = nonf['nflId'].astype(str)

tackles['gameId'] = tackles['gameId'].astype(str)
tackles['playId'] = tackles['playId'].astype(str)
tackles['nflId'] = tackles['nflId'].astype(str)

# Step 2: Perform the merge
nonf = pd.merge(nonf, tackles[['gameId', 'playId', 'nflId', 'tackle']],
                on=['gameId', 'playId', 'nflId'],
                how='left')

# Replace NaN in the 'tackle' column with 0
nonf['tackle'].fillna(0, inplace=True)

# Convert 'tackle' to integer
nonf['tackle'] = nonf['tackle'].astype(int)


In [ ]:
only1 = filtered_df[filtered_df['overlapRank']==1]

In [ ]:
import pandas as pd

# Assuming only1 and nonf are your DataFrames

# Step 1: Check data types and convert if necessary
# Convert gameId, playId, and frameId to string if they are not already
only1['gameId'] = only1['gameId'].astype(str)
only1['playId'] = only1['playId'].astype(str)
only1['frameId'] = only1['frameId'].astype(str)
nonf['gameId'] = nonf['gameId'].astype(str)
nonf['playId'] = nonf['playId'].astype(str)
nonf['frameId'] = nonf['frameId'].astype(str)

# Also ensure playerName and displayName are of the same type
only1['playerName'] = only1['playerName'].astype(str)
nonf['displayName'] = nonf['displayName'].astype(str)

# Step 2: Perform the merge
merged_df = pd.merge(only1, nonf, left_on=['gameId', 'playId', 'frameId', 'playerName'],
                     right_on=['gameId', 'playId', 'frameId', 'displayName'], how='left')




In [ ]:
merged_df['standardized_difference'] = abs(merged_df['ballcarrier_difference']/90)

In [ ]:
import pandas as pd

# Convert 'overlapPercentage' to numeric, coerce any non-numeric values to NaN
merged_df['overlapPercentage'] = pd.to_numeric(merged_df['overlapPercentage'], errors='coerce')

# Create the 'is_overlap' column based on whether 'overlapPercentage' is greater than 0
# NaN values will be treated as False in this comparison
merged_df['is_overlap'] = (merged_df['overlapPercentage'] > 0).astype(int)

# Display the updated DataFrame



In [ ]:
merged_df['set_score'] = ((merged_df['overlapPercentage']/100) + merged_df['standardized_difference'])/2 * merged_df['is_inside']

In [ ]:
only_overlap = merged_df[merged_df['is_overlap']==1]

In [ ]:
import pandas as pd

# Assuming only_overlap is your DataFrame

# Perform the groupby and aggregation
stats_df = only_overlap.groupby(['gameId', 'playId', 'playerName']).agg({
    'set_score': ['sum', 'mean'],
    'position_x': 'first',
    'expectedPointsAdded_x': 'first',
    'passProbability_x': 'first',
    'playResult_x': 'first',
    'overlapPercentage':'first',
    'standardized_difference':'first',
    'tackle':'first',
}).reset_index()

# Flatten the MultiIndex columns
stats_df.columns = ['_'.join(col).strip('_') for col in stats_df.columns.values]

# Rename the columns for clarity
stats_df.rename(columns={
    'set_tackle_sum': 'cumulative_set_tackle',
    'set_tackle_mean': 'average_set_tackle',
    'position_x_first': 'position_x',
    'epa_first': 'expectedPointsAdded_x',
    'pass_probability_x_first': 'passProbability_x',
    'playResult_x_first': 'playResult_x'
}, inplace=True)

# Display the new


In [ ]:
import pandas as pd

# Assuming stats_df is your DataFrame

# Function to modify set_score_sum and set_score_mean based on tackle_first
def modify_scores(row):
    if row['tackle_first'] == 1:
        row['set_score_sum_incl_tackle'] = row['set_score_sum'] + 0.5
        row['set_score_mean_incl_tackle'] = row['set_score_mean'] + 0.5
    else:
        row['set_score_sum_incl_tackle'] = row['set_score_sum']
        row['set_score_mean_incl_tackle'] = row['set_score_mean']
    return row

# Apply the function to the DataFrame
stats_df = stats_df.apply(modify_scores, axis=1)

# Display the updated DataFrame


In [ ]:
# Assuming 'set_score_mean_incl_tackle' is a column in your existing DataFrame named stats_df
stats_df['successful'] = (stats_df['set_score_mean_incl_tackle'] > 0).astype(int)


In [ ]:
import pandas as pd

# Assuming stats_df_sorted is your DataFrame

# Identify categorical and numerical columns
categorical_cols = stats_df.select_dtypes(include=['object', 'category']).columns
numerical_cols = stats_df.select_dtypes(include=['number']).columns

# Define aggregation logic
aggregation_logic = {col: 'first' for col in categorical_cols if col != 'playerName'}  # Keep first value for categorical columns, except 'playerName'
aggregation_logic.update({col: 'mean' for col in numerical_cols if col not in ['cumulative_set_tackle', 'set_score_sum_incl_tackle', 'set_score_mean_incl_tackle']})  # Average for numerical, except specified columns

aggregation_logic['set_score_sum_incl_tackle'] = 'sum'  # Sum for 'set_score_sum_incl_tackle'
aggregation_logic['set_score_mean_incl_tackle'] = 'mean'  # Mean for 'set_score_mean_incl_tackle'

# Add a column to count the number of plays per 'gameId' and 'playerName' group
stats_df['num_plays'] = stats_df.groupby(['gameId', 'playerName'])['playId'].transform('nunique')
aggregation_logic['num_plays'] = 'first'  # Keep the first value of num_plays, which is the same for all rows in the group

# Group by 'gameId' and 'playerName', then apply aggregation
grouped_df = stats_df.groupby(['gameId', 'playerName'], as_index=False).agg(aggregation_logic)

# Display the DataFrame


# Display the Dat

In [ ]:
grouped_df.head()

,playerName,gameId,playId,position_x,set_score_sum,set_score_mean,expectedPointsAdded_x_first,passProbability_x_first,playResult_x,overlapPercentage_first,standardized_difference_first,tackle_first,num_plays,set_score_percentile,playResult_x_jittered,passProbability_x_jittered,successful,epa_jittered,set_score_sum_incl_tackle,set_score_mean_incl_tackle
0,Cameron Jordan,2022102000,1256,DE,0.614122,0.087732,0.021564,0.446337,9.666667,1.125330e+01,0.105037,0.0,3,0.462121,9.663855,0.021209,0.333333,0.020983,1.842367,0.087732
1,Carl Granderson,2022102000,1454,DE,0.000000,0.000000,2.325146,0.360233,2.000000,1.770701e-16,0.051667,0.0,1,0.229545,2.017684,2.332706,0.000000,2.316543,0.000000,0.000000
2,Chris Harris,2022102000,1206,CB,0.000000,0.000000,-0.253301,0.613955,3.000000,7.455418e+00,0.256889,0.0,1,0.229545,3.012552,-0.240679,0.000000,-0.248222,0.000000,0.000000
3,J.J. Watt,2022102000,2031,DE,0.919190,0.070707,-0.808164,0.317174,-1.000000,1.618618e+00,0.222444,0.0,2,0.497727,-0.999080,-0.801070,0.500000,-0.807601,1.838380,0.070707
4,Marco Wilson,2022102000,1597,CB,1.305732,0.217622,0.133769,0.559562,10.000000,3.915342e+00,0.382778,0.0,1,0.881818,10.012062,0.132462,1.000000,0.131552,1.305732,0.217622


In [ ]:
grouped_df = grouped_df.sort_values(by='set_score_sum_incl_tackle', ascending=False)

In [ ]:
grouped_df.head()

,playerName,gameId,playId,position_x,set_score_sum,set_score_mean,expectedPointsAdded_x_first,passProbability_x_first,playResult_x,overlapPercentage_first,standardized_difference_first,tackle_first,num_plays,set_score_sum_incl_tackle,set_score_mean_incl_tackle
88,James Smith-Williams,2022102306,1531,DE,2.814580,0.127935,-0.833501,0.481511,-5.333333,9.265618,0.803704,0.0,3,8.443740,0.127935
107,Patrick Surtain,2022102307,1834,CB,1.304125,0.130413,-0.268561,0.451860,2.333333,16.372111,0.178481,0.0,3,3.912376,0.130413
119,Rasheem Green,2022102308,1063,DE,1.898961,0.067820,0.201101,0.601645,3.500000,6.981787,0.518722,0.0,2,3.797921,0.067820
54,Anthony Brown,2022102303,1700,CB,0.935577,0.077965,0.353866,0.466975,8.750000,6.837085,0.185222,0.0,4,3.742307,0.077965
127,Khalil Mack,2022102309,109,OLB,0.340858,0.017940,-0.064796,0.441540,5.000000,5.131480,0.354178,0.0,10,3.408581,0.017940


In [ ]:
import pandas as pd

# Assuming grouped_df is your DataFrame

# Function to scale percentile rank to 1-99 range
def scale_percentile(series):
    percentile_rank = series.rank(pct=True)  # Calculate percentile rank
    scaled_percentile = percentile_rank * 99  # Scale to 99 instead of 100
    return scaled_percentile.round().clip(lower=1)  # Round and clip to ensure the range is 1 to 99

# Calculate scaled percentile for 'cumulative_set_tackle'
grouped_df['cumulative_set_tackle_percentile'] = (grouped_df['set_score_sum_incl_tackle'].rank(pct=True) * 100).round()

# Calculate scaled percentile for 'average_set_tackle'
grouped_df['average_set_tackle_percentile'] = (grouped_df['set_score_mean_incl_tackle'].rank(pct=True) * 100).round()

# Calculate the absolute value of 'standardized_difference_first'
grouped_df['standardized_difference_first_abs'] = grouped_df['standardized_difference_first'].abs()

# Calculate scaled percentile for 'overlapPercentage_first'
grouped_df['overlapPercentage_percentile'] = scale_percentile(grouped_df['overlapPercentage_first'])

# Calculate scaled percentile for the absolute value of 'standardized_difference_first'
grouped_df['standardized_difference_percentile'] = scale_percentile(grouped_df['standardized_difference_first_abs'])

# Display the updated DataFrame




In [ ]:
grouped_df.head()

,playerName,gameId,playId,position_x,set_score_sum,set_score_mean,expectedPointsAdded_x_first,passProbability_x_first,playResult_x,overlapPercentage_first,standardized_difference_first,tackle_first,num_plays,set_score_percentile,playResult_x_jittered,passProbability_x_jittered,successful,epa_jittered,set_score_sum_incl_tackle,set_score_mean_incl_tackle,cumulative_set_tackle_percentile,average_set_tackle_percentile,standardized_difference_first_abs,overlapPercentage_percentile,standardized_difference_percentile
93,Preston Smith,2022102306,1061,OLB,1.749504,0.106573,-0.416861,0.505061,2.000000,3.883270,0.116211,0.0,10,0.594318,1.996791,-0.416853,0.700000,-0.413853,17.495039,0.106573,100.0,68.0,0.116211,42.0,35.0
28,Brian Burns,2022102301,123,OLB,7.190094,0.268113,-1.535662,0.780496,-3.500000,6.772897,0.164556,0.0,2,0.843182,-3.511457,-1.538344,1.000000,-1.543978,14.380187,0.268113,99.0,98.0,0.164556,66.0,62.0
88,James Smith-Williams,2022102306,1531,DE,4.280739,0.242738,-0.833501,0.481511,-5.333333,9.265618,1.303741,0.0,3,0.712121,-5.331713,-0.835508,0.666667,-0.833190,12.842218,0.242738,99.0,96.0,1.303741,82.0,99.0
67,Jihad Ward,2022102304,2812,OLB,1.520829,0.115520,-0.096730,0.480549,4.142857,4.430035,0.159111,0.0,7,0.631818,4.143806,-0.096354,0.714286,-0.096872,10.645802,0.115520,98.0,72.0,0.159111,52.0,57.0
94,Rashan Gary,2022102306,1253,OLB,1.487722,0.105523,0.184939,0.592125,8.428571,1.194762,0.203524,0.0,7,0.598701,8.430911,0.188300,0.714286,0.184402,10.414051,0.105523,98.0,67.0,0.203524,17.0,77.0


In [ ]:
import pandas as pd

# Assuming grouped_df is your DataFrame
# Assuming the percentile column for cumulative percent is named 'cumulative_set_tackle_percentile'

# Filter and get top 10 for CB
top10_CB = grouped_df[grouped_df['position_x'] == 'CB'].nlargest(25, 'cumulative_set_tackle_percentile')

# Filter and get top 10 for OLB
top10_OLB = grouped_df[grouped_df['position_x'] == 'OLB'].nlargest(25, 'cumulative_set_tackle_percentile')

# Filter and get top 10 for DE
top10_DE = grouped_df[grouped_df['position_x'] == 'DE'].nlargest(25, 'cumulative_set_tackle_percentile')


